In [1]:
#Dependencies
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create a reference the CSV file desired
csv_path = "FARS2016NationalCSV/accident.csv"

# Read the CSV into a Pandas DataFrame
accidents = pd.read_csv(csv_path)

In [3]:
# drop the columns that are not needed
accidents_df = accidents.drop(accidents.columns[[2,4,5,6,7,8,17,18,19,20,24,27,30,31,32,33,39,40,41,42,43,44,45,46]], axis=1)



In [4]:
# get path of csv's with code and their description
csv_path2 = "FARS2016NationalCSV/weather.csv"
csv_path3 = "FARS2016NationalCSV/CF.csv"
csv_path4 = "FARS2016NationalCSV/light.csv"
csv_path5 = "FARS2016NationalCSV/road.csv"
csv_path6 = "FARS2016NationalCSV/Route.csv"
csv_path7 = "FARS2016NationalCSV/Man_Coll.csv"
csv_path8 = "FARS2016NationalCSV/Harm_Ev.csv"

# read all those csv file
weather = pd.read_csv(csv_path2)
cf = pd.read_csv(csv_path3)
light_cond = pd.read_csv(csv_path4)
road = pd.read_csv(csv_path5)
route = pd.read_csv(csv_path6)
man_coll = pd.read_csv(csv_path7)
harm_ev = pd.read_csv(csv_path8)

# replace the codes with description
accidents_df['WEATHER'] = accidents_df['WEATHER'].map(weather.set_index('Code')['Weather'])
accidents_df['WEATHER1'] = accidents_df['WEATHER1'].map(weather.set_index('Code')['Weather'])
accidents_df['WEATHER2'] = accidents_df['WEATHER2'].map(weather.set_index('Code')['Weather'])
accidents_df['CF1'] = accidents_df['CF1'].map(cf.set_index('Code')['Attribute'])
accidents_df['CF2'] = accidents_df['CF2'].map(cf.set_index('Code')['Attribute'])
accidents_df['CF3'] = accidents_df['CF3'].map(cf.set_index('Code')['Attribute'])
accidents_df['LGT_COND'] = accidents_df['LGT_COND'].map(light_cond.set_index('Code')['Attribute'])
accidents_df['REL_ROAD'] = accidents_df['REL_ROAD'].map(road.set_index('Code')['Attribute'])
accidents_df['MAN_COLL'] = accidents_df['MAN_COLL'].map(man_coll.set_index('Code')['Attribute'])
accidents_df['HARM_EV'] = accidents_df['HARM_EV'].map(harm_ev.set_index('Code')['Attribute'])
accidents_df['ROUTE'] = accidents_df['ROUTE'].map(route.set_index('Code')['Attribute'])

In [5]:
accidents_df.head()

,STATE,ST_CASE,VE_FORMS,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,...,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR
0,1,10001,1,73,1716,1,1,2016,6,9,...,Outside Trafficway,Daylight,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
1,1,10002,1,73,1716,10,1,2016,1,19,...,On Roadside,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
2,1,10003,1,73,0,1,1,2016,6,2,...,On Roadside,Dark - Lighted,Cloudy,No Additional Atmospheric Conditions,Cloudy,None,None,None,1,0
3,1,10004,1,73,0,17,1,2016,1,21,...,On Median,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
4,1,10005,1,73,3125,26,1,2016,3,20,...,Outside Trafficway,Dark - Lighted,Rain,No Additional Atmospheric Conditions,Rain,None,None,None,1,0


In [6]:

# get State Names from State Codes
csv_path1 = "FARS2016NationalCSV/states.csv"
states = pd.read_csv(csv_path1)
accidents_df['STATE'] = accidents_df['STATE'].map(states.set_index('Codes')['State'])

#Load City and County code from GLC.xlsx (downloaded from GSA.gov)
xls_path = "FARS2016NationalCSV/GLC.xlsx"
xl = pd.ExcelFile(xls_path)
state_codes = xl.parse("Sheet 1")

# cleaning up file
state_codes.fillna(0, inplace=True)
state_codes["City Code"]= state_codes["City Code"].astype(int)
state_codes["County Code"]= state_codes["County Code"].astype(int)
state_codes["State Name"] = state_codes["State Name"].astype(str) 

state_codes = state_codes.apply(lambda x: x.astype(str).str.title())

# create unique index to main dataframe and state_codes dataframe in order to map City Names
state_codes["state_county_city"] = state_codes["State Name"].map(str) + "_" + state_codes["County Code"].map(str) + "_" + state_codes["City Code"].map(str)
state_codes["state_county"] = state_codes["State Name"].map(str) + "_" + state_codes["County Code"].map(str) + "_" + "0"
accidents_df["state_county_city"] = accidents_df["STATE"].map(str) + "_" + accidents_df["COUNTY"].map(str) + "_" + accidents_df["CITY"].map(str)
accidents_df["state_county"] = accidents_df["STATE"].map(str) + "_" + accidents_df["COUNTY"].map(str) + "_" + "0"

# get rid of duplicates
state_codes.loc[state_codes.duplicated('state_county_city', keep=False)]
state_codes = state_codes.drop(state_codes.index[[1049,46088]])

# map City Names to main dataframe
accidents_df['CITY'] = accidents_df['state_county_city'].map(state_codes.set_index('state_county_city')['City Name/County Name'])
accidents_df['COUNTY'] = accidents_df['state_county'].map(state_codes.set_index('state_county_city')['City Name/County Name'])




In [7]:
# Delete "unique index used to populate County and City"
accidents_df = accidents_df.drop(accidents_df.columns[[28,29]], axis=1)

In [8]:
# visualize data with county_city
accidents_df.head()

,STATE,ST_CASE,VE_FORMS,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,...,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR
0,Alabama,10001,1,Jefferson,Hoover,1,1,2016,6,9,...,Outside Trafficway,Daylight,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
1,Alabama,10002,1,Jefferson,Hoover,10,1,2016,1,19,...,On Roadside,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
2,Alabama,10003,1,Jefferson,Jefferson,1,1,2016,6,2,...,On Roadside,Dark - Lighted,Cloudy,No Additional Atmospheric Conditions,Cloudy,None,None,None,1,0
3,Alabama,10004,1,Jefferson,Jefferson,17,1,2016,1,21,...,On Median,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
4,Alabama,10005,1,Jefferson,Vestavia Hills,26,1,2016,3,20,...,Outside Trafficway,Dark - Lighted,Rain,No Additional Atmospheric Conditions,Rain,None,None,None,1,0


In [9]:
accidents_df.columns

Index(['STATE', 'ST_CASE', 'VE_FORMS', 'COUNTY', 'CITY', 'DAY', 'MONTH',
       'YEAR', 'DAY_WEEK', 'HOUR', 'MINUTE', 'ROUTE', 'TWAY_ID', 'TWAY_ID2',
       'LATITUDE', 'LONGITUD', 'HARM_EV', 'MAN_COLL', 'REL_ROAD', 'LGT_COND',
       'WEATHER1', 'WEATHER2', 'WEATHER', 'CF1', 'CF2', 'CF3', 'FATALS',
       'DRUNK_DR'],
      dtype='object')

In [10]:
# add columns for date and time 
# add column to distinguish betweent Holidays/Weekends and Weekdays
accidents_df["DATE"] = pd.to_datetime(accidents_df[["DAY","MONTH","YEAR","HOUR","MINUTE"]])
accidents_df["TIME"] = accidents_df["DATE"].dt.time
accidents_df['HOLIDAYS'] = np.where((accidents_df["DATE"] > '2016-01-01') & (accidents_df["DATE"] < '2016-01-04 06:00:00')|
                                ((accidents_df["DATE"] > '2016-05-27 17:59:00') & (accidents_df["DATE"] < '2016-05-31 06:00:00'))|
                                ((accidents_df["DATE"] > '2016-07-01 17:59:00') & (accidents_df["DATE"] < '2016-07-05 06:00:00'))|
                                ((accidents_df["DATE"] > '2016-09-02 17:59:00') & (accidents_df["DATE"] < '2016-09-06 06:00:00'))|
                                ((accidents_df["DATE"] > '2016-11-23 17:59:00') & (accidents_df["DATE"] < '2016-11-28 06:00:00'))|
                                ((accidents_df["DATE"] > '2016-12-23 17:59:00') & (accidents_df["DATE"] < '2016-12-27 06:00:00'))|
                                ((accidents_df["DAY_WEEK"] == 6) & (accidents_df["HOUR"] >= 18) & (accidents_df["HOUR"] <= 23))|
                                (accidents_df["DAY_WEEK"] == 1)|(accidents_df["DAY_WEEK"] == 7)|
                                ((accidents_df["DAY_WEEK"] == 2) & (accidents_df["HOUR"] >= 0) & (accidents_df["HOUR"] <= 5))
                                , "1","0")

In [11]:
accidents_df.to_csv("accidents-final.csv")

In [12]:
# get the data from accidents.csv for the year 2015 to analyze the most harmful events for the year 2015
csv_path_2015 = "FARS2016NationalCSV/accident_2015.csv"
accident_2015 = pd.read_csv(csv_path_2015)
# drop all columns other than ST_CASE and HARM_EV
accident_2015 = accident_2015.drop(accident_2015.columns[[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1)
accident_2015['HARM_EV'] = accident_2015['HARM_EV'].map(harm_ev.set_index('Code')['Attribute'])
accident_2015 = accident_2015.rename(columns={'HARM_EV': 'HARM_EV_2015'})
accident_2015.head()


,ST_CASE,HARM_EV_2015
0,10001,Embankment
1,10002,Ditch
2,10003,Tree (Standing Only)
3,10004,Mail Box
4,10005,Motor Vehicle In-Transport


In [13]:
accident_2015.to_csv("accidents-2015.csv")